In [4]:
import pyscreenshot as ImageGrab
import os
import time
import webbrowser
import pyautogui
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import Xlib.threaded

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [6]:
import socket, cv2
from datetime import datetime
import threading

In [7]:
from selenium.webdriver.common.action_chains import ActionChains

In [55]:
# Global variables
# -------------------------------

start = os.getcwd()
game_path = "file://" + start + "/Entorno/mk.js-master/game/index.html"

# https://keycode.info/
# https://pyautogui.readthedocs.io/en/latest/keyboard.html

p1_controls_keynum = {
    "RIGHT" : 74,
    "LEFT" : 71,
    "UP" : 89,
    "DOWN" : 72,
    "BLOCK" : 81,
    "HIGH_PUNCH" : 65,
    "LOW_PUNCH" : 83,
    "LOW_KICK" : 68,
    "HIGH_KICK" : 70
}

p1_controls = {
    "RIGHT" : 'j',
    "LEFT" : 'g',
    "UP" : 'y',
    "DOWN" : 'h',
    "BLOCK" : 'q',
    "HIGH_PUNCH" : 'a',
    "LOW_PUNCH" : 's',
    "LOW_KICK" : 'd',
    "HIGH_KICK" : 'f'
}

p1_controls_selenium = {
    "RIGHT" : 'j',
    "LEFT" : 'g',
    "UP" : 'y',
    "DOWN" : 'h',
    "BLOCK" : 'q',
    "HIGH_PUNCH" : 'a',
    "LOW_PUNCH" : 's',
    "LOW_KICK" : 'd',
    "HIGH_KICK" : 'f'
}

p2_controls_keynum = {
    "RIGHT" : 39,
    "LEFT" : 37,
    "UP" : 38,
    "DOWN" : 40,
    "BLOCK" : 85,
    "HIGH_PUNCH" : 80,
    "LOW_PUNCH" : 79,
    "LOW_KICK" : 76,
    "HIGH_KICK" : 220
}

p2_controls = {
    "RIGHT" : 'right',
    "LEFT" : 'left',
    "UP" : 'up',
    "DOWN" : 'down',
    "BLOCK" : 'u',
    "HIGH_PUNCH" : 'p',
    "LOW_PUNCH" : 'o',
    "LOW_KICK" : 'l',
    "HIGH_KICK" : 'k'
}

p2_controls_selenium = {
    "RIGHT" : u'\ue014',
    "LEFT" : u'\ue012',
    "UP" : u'\ue013',
    "DOWN" : u'\ue015',
    "BLOCK" : 'u',
    "HIGH_PUNCH" : 'p',
    "LOW_PUNCH" : 'o',
    "LOW_KICK" : 'l',
    "HIGH_KICK" : 'k'
}

translations = ["RIGHT", "LEFT", "UP", "DOWN", "BLOCK", "HIGH_PUNCH", "LOW_PUNCH", "LOW_KICK", "HIGH_KICK"]

In [43]:
def setHealthBarPos(x_im, y_im):
    coordinate = x, y = -1, -1
    green = (60, 212, 0)
    brown = (185, 122, 87)
    found = False
    im = screenGrab(x_im, y_im)
    while not found and (coordinate[0]<598):
        coordinate = x, y = coordinate[0]+1, -1
        while not found and (coordinate[1]<393): 
            coordinate = x, y = coordinate[0], coordinate[1]+1
            pixel = im.getpixel(coordinate)
            if pixel == green:
                found = True
    #now we have the p1_x and the y, we can guess the position in which p2_x ends
    p1_x = coordinate[0]
    pos_y = coordinate[1]
    p2_x = 599
    #now we extract the length and start point of p2's health
    length = 0
    coord = x, y = 598, pos_y
    change = 0
    previous = im.getpixel(coord)
    coord = x, y = 597, pos_y
    while not (change == 2):
        if (im.getpixel(coord) != previous):
            change = change+1
            previous = im.getpixel(coord)
            if (change==1):
                length = 1
                p2_x = coord[0]
        else:
            if (change==1):
                length= length+1
        coord = x, y = coord[0]-1, pos_y
    p2_x = p2_x - length;
    print("P1_X: " + str(p1_x) + " P2_X: "+str(p2_x)+" LENGTH: " +str(length))
    return (p1_x, p2_x, y, length)
    

def setPadding():
    # Buscar el primer pixel que corresponda al color de fondo
    # marron = #B97A57     rgb(185,122,87)
    im = ImageGrab.grab()
    width, height = im.size
    coordinate = x, y = 0, 0
    found = False
    while not found and (coordinate[0]<width):
        while not found and (coordinate[1]<height): 
            pixel = im.getpixel(coordinate)
            #windows = (185, 122, 87)
            #linux = (188,120,83)
            if pixel == (188,120,83):
                print(coordinate)
                p1_x, p2_x, y, length = setHealthBarPos(coordinate[0], coordinate[1])
                return (coordinate[0], coordinate[1], p1_x, p2_x, y, length)
                found = True
            coordinate = x, y = coordinate[0], coordinate[1]+1
        coordinate = x, y = coordinate[0]+1, 0

def screenGrab(x_pad, y_pad): #Captures a section of the screen where the play area is
    box = (x_pad, y_pad, x_pad+599, y_pad+394)
    im = ImageGrab.grab(box)
    return im

In [44]:
def setPadding_next(x_pad, y_pad):
    #Buscar el segundo panel de juego
    #Se encuentra verticalmente bajo el primero, por tanto solo es necesario encontrar la distancia vertical y sumarla a lo previo
    #Exclusivamente necesario para el servidor cuando toma muestras del entorno
    im = ImageGrab.grab()
    width, height = im.size
    coordinate = x, y = x_pad, y_pad+398
    found = False
    while not found and (coordinate[1]<height):
        pixel = im.getpixel(coordinate)
        if (pixel <= (188,120,83)) and (pixel >= (180,115,80)):
            p1_x, p2_x, y, length = setHealthBarPos(coordinate[0], coordinate[1])
            return (coordinate[0], coordinate[1], p1_x, p2_x, y, length)
            found = True
        coordinate = x, y = coordinate[0], coordinate[1]+1

In [45]:
def testBrowser():
    if (os.name == 'posix'):
        #path="/home/tai/Desktop/TFG/geckodriver"
        path = start + "/geckodriver"
        driver = webdriver.Firefox(executable_path=path)
    else:
        driver = webdriver.Firefox()
    driver.get(game_path)
    return driver

def refreshEnv(driv):
    driv.refresh()

In [46]:
def startBrowser_sized(pos):
    if (os.name == 'posix'):
        #path="/home/tai/Desktop/TFG/geckodriver"
        path = start + "/geckodriver"
        driver = webdriver.Firefox(executable_path=path)
    else:
        driver = webdriver.Firefox()
    driver.get(game_path)
    driver.set_window_size(725, 550)
    driver.set_window_position(0,(460*pos))
    return driver

In [47]:
def translate(command_representation):
    res = []
    for x in command_representation:
        res.append(translations[int(x)])
    return res

In [48]:
def inputControl(command_representation, player):
    command = translate(command_representation) #In case the output of the ai isn't on the parameters we have for inputs
    if (player==1):
        if (len(command)==2):
            pyautogui.hotkey(p1_controls[command[0]], p1_controls[command[1]])
        else: #SOLO SE PERMITE 1 O 2 TECLAS AL MISMO TIEMPO
            pyautogui.press(p1_controls[command[0]])
    else:
        if (len(command)==2):
            pyautogui.hotkey(p2_controls[command[0]], p2_controls[command[1]])
        else: #SOLO SE PERMITE 1 O 2 TECLAS AL MISMO TIEMPO
            pyautogui.press(p2_controls[command[0]])
            
def inputControl_Selenium(command_representation, player, driver):
    command = translate(command_representation) #In case the output of the ai isn't on the parameters we have for inputs
    if (player==1):
        if (len(command)==2):
            ActionChains(driver) \
            .key_down(p1_controls_selenium[command[0]]) \
            .key_down(p1_controls_selenium[command[1]]) \
            .key_up(p1_controls_selenium[command[0]]) \
            .key_up(p1_controls_selenium[command[1]]) \
            .perform()
        else:
            ActionChains(driver) \
            .key_down(p1_controls_selenium[command[0]]) \
            .key_up(p1_controls_selenium[command[0]]) \
            .perform()
    else:
        if (len(command)==2):
            ActionChains(driver) \
            .key_down(p2_controls_selenium[command[0]]) \
            .key_down(p2_controls_selenium[command[1]]) \
            .key_up(p2_controls_selenium[command[0]]) \
            .key_up(p2_controls_selenium[command[1]]) \
            .perform()
        else:
            ActionChains(driver) \
            .key_down(p2_controls_selenium[command[0]]) \
            .key_up(p2_controls_selenium[command[0]]) \
            .perform()

In [49]:
def setTime():
    dt = datetime.now()
    if (dt.second % 10 > 5):
        sec = (((dt.second // 10)+1)*10) + 5 #Z+1 5
        dt.replace(second=sec)
    else:
        sec = ((dt.second // 10)+1)*10 #Z+1 0
        dt.replace(second=sec)
    return dt.strftime("%Y %b %H:%M:%S")

In [50]:
def isGreen(rgb):
    r, g, b = rgb
    if (g>r) and (g>b):
        return True
    return False

def isBlue(rgb):
    r, g, b = rgb
    if (b>r) and (b>g):
        return True
    return False

def isRed(rgb):
    r, g, b = rgb
    if (r>b) and (r>g):
        return True
    return False

In [51]:
def printHealths(im, coord_p1, coord_p2, right_end):
    p1 = 0
    p2 = 0
    #verde es #3CD400 == rgb(60,212,0) aproximadamente, isGreen comprueba que G sea el valor principal
    for x in range(right_end+1):
        if (isGreen(im.getpixel(coord_p1))):
            p1 = p1 + 1
        if (isGreen(im.getpixel(coord_p2))):
            p2 = p2 + 1
        coord_p1 = x, y = coord_p1[0]+1, coord_p1[1]
        coord_p2 = x, y = coord_p2[0]+1, coord_p2[1]
    print("Salud P1: " + str(p1))
    print("Salud P2: " + str(p2))
    return(p1,p2)

In [52]:
def listener(driver, x_pad, y_pad, p1_x, p2_x, y, length, port, start_p1, start_p2):
    control = [start_p1, start_p2]
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('', port))
    s.listen(5)

    while True:
        dialogo, address = s.accept()
        mensaje = dialogo.recv(1024).decode("utf-8")
        if(mensaje=="get"): #send image
            im = screenGrab(x_pad, y_pad)
            data = np.array(im)
            stringData = data.tostring()
            dialogo.send(str.encode(str(im.width).ljust(8)))
            dialogo.send(str.encode(str(im.height).ljust(8)))
            dialogo.send(str.encode(str(len(stringData)).ljust(16)));
            dialogo.send( stringData );
            control = [0,0]
        elif(mensaje=="set"): #send information, health padding
            a = str(p1_x)+";"+str(p2_x)+";"+str(y)+";"+str(length)
            dialogo.sendall(str.encode(a.ljust(20)))
            control = [0,0]
        elif (mensaje == "start1"):
            if (control[0] == 0 or control[1] == 0):
                control[0] = 1
                a = "No"
            #a = setTime()
                dialogo.sendall(str.encode(a.ljust(20)))
            elif (control[0] == 1 and control[1] >=1):
                control[0] = 2
                a = "Yes"
                dialogo.sendall(str.encode(a.ljust(20)))
                refreshEnv(driver)
                #pressf5()
            else:
                a = "No"
                dialogo.sendall(str.encode(a.ljust(20)))
        elif (mensaje == "start2"):
            if (control[0] == 0 or control[1] == 0):
                control[1] = 1
                a = "No"
            #a = setTime()
                dialogo.sendall(str.encode(a.ljust(20)))
            elif (control[0] >= 1 and control[1] ==1):
                control[1] = 2
                a = "Yes"
                dialogo.sendall(str.encode(a.ljust(20)))
                refreshEnv(driver)
                #pressf5()
            else:
                a = "No"
                dialogo.sendall(str.encode(a.ljust(20)))
        else:   #el resto de mensajes van a ser inputs <player><command>
            inputControl_Selenium(mensaje[1:], int(mensaje[0]), driver)
            control = [0,0]
        dialogo.close() 
    s.close()

## Multi-entorno paralelizado
Puertos: [50010 .. 50013]

In [56]:
def main():
    aux_driver2 = startBrowser_sized(1)
    aux_driver1 = startBrowser_sized(0)
    
    drivers = [aux_driver1, aux_driver2]
    
    time.sleep(1)  #give a little time to set everything
    
    (x_pad1, y_pad1, p1_x1, p2_x1, y1, length1) = setPadding()
    (x_pad2, y_pad2, p1_x2, p2_x2, y2, length2)= setPadding_next(x_pad1, y_pad1)
    
    x_pad = [x_pad1, x_pad2]
    y_pad = [y_pad1, y_pad2]
    p1_x = [p1_x1, p1_x2]
    p2_x = [p2_x1, p2_x2]
    y = [y1, y2]
    length = [length1, length2]

    pyautogui.FAILSAFE = False #Esto debería bloquear que nos de error en el pyautogui
    PORT = 50010
    start_p1 = [0, 0]
    start_p2 = [0, 0]
    
    #se inician procesos paralelos que controlan cada entorno
    env1_1 = lambda: listener(drivers[0], x_pad[0], y_pad[0], p1_x[0], p2_x[0], y[0], length[0], PORT, start_p1[0], start_p2[0])
    t1_1 = threading.Thread(target=(env1_1))
    t1_1.start()
    print(PORT)
    time.sleep(0.5)
    env1_2 = lambda: listener(drivers[0], x_pad[0], y_pad[0], p1_x[0], p2_x[0], y[0], length[0], PORT+1, start_p1[0], start_p2[0])
    t1_2 = threading.Thread(target=(env1_2))
    t1_2.start()
    print(PORT+1)
    time.sleep(0.5)
    env2_1 = lambda: listener(drivers[1], x_pad[1], y_pad[1], p1_x[1], p2_x[1], y[1], length[1], PORT+2, start_p1[1], start_p2[1])
    t2_1 = threading.Thread(target=(env2_1))
    t2_1.start()
    print(PORT+2)
    time.sleep(0.5)
    env2_2 = lambda: listener(drivers[1], x_pad[1], y_pad[1], p1_x[1], p2_x[1], y[1], length[1], PORT+3, start_p1[1], start_p2[1])
    t2_2 = threading.Thread(target=(env2_2))
    t2_2.start()
    print(PORT+3)
    
if __name__ == '__main__':
    main()

(78, 190)
P1_X: 17 P2_X: 376 LENGTH: 200
P1_X: 17 P2_X: 376 LENGTH: 200
50010
50011
50012
50013


## Un solo entorno paralelizado
Puertos: [50005 .. 50006]

In [17]:
def main():
    driver = startBrowser_sized(0)
    time.sleep(1)  #give a little time to set everything
    (x_pad, y_pad, p1_x, p2_x, y, length) = setPadding() #get the padding info
    pyautogui.FAILSAFE = False
    PORT = 50005
    start_p1 = 0
    start_p2 = 0
    
    env_1 = lambda: listener(driver, x_pad, y_pad, p1_x, p2_x, y, length, PORT, start_p1, start_p2)
    t_1 = threading.Thread(target=(env_1))
    t_1.start()
    print(PORT)
    time.sleep(0.5)
    env_2 = lambda: listener(driver, x_pad, y_pad, p1_x, p2_x, y, length, PORT+1, start_p1, start_p2)
    t_2 = threading.Thread(target=(env_2))
    t_2.start()
    print(PORT+1)

if __name__ == '__main__':
    main()

(78, 190)
P1_X: 17 P2_X: 376 LENGTH: 200
50005
50006


## Servidor multientorno no paralelizado
Puertos: [50018 .. 50021]

In [ ]:
def main():
    aux_driver2 = startBrowser_sized(1)
    aux_driver1 = startBrowser_sized(0)
    
    drivers = [aux_driver1, aux_driver2]
    
    time.sleep(1)
    
    (x_pad1, y_pad1, p1_x1, p2_x1, y1, length1) = setPadding()
    (x_pad2, y_pad2, p1_x2, p2_x2, y2, length2)= setPadding_next(x_pad1, y_pad1)
    
    x_pad = [x_pad1, x_pad2]
    y_pad = [y_pad1, y_pad2]
    p1_x = [p1_x1, p1_x2]
    p2_x = [p2_x1, p2_x2]
    y = [y1, y2]
    length = [length1, length2]
    
    pyautogui.FAILSAFE = False
    PORT = 50018
    start_p1 = [0, 0]
    start_p2 = [0, 0]
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('', PORT))
    s.listen(5)

    while True:
        dialogo, address = s.accept()
        mensaje = dialogo.recv(1024).decode("utf-8")
        if(mensaje[1:]=="get"): #send image
            im = screenGrab(x_pad[int(mensaje[0])], y_pad[int(mensaje[0])])
            data = np.array(im)
            stringData = data.tostring()
            dialogo.send(str.encode(str(im.width).ljust(8)))
            dialogo.send(str.encode(str(im.height).ljust(8)))
            dialogo.send(str.encode(str(len(stringData)).ljust(16)));
            dialogo.send( stringData );
            start_p1[int(mensaje[0])] = 0
            start_p2[int(mensaje[0])] = 0
        elif(mensaje[1:]=="set"): #send information, health padding
            a = str(p1_x[int(mensaje[0])])+";"+str(p2_x[int(mensaje[0])])+";"+str(y[int(mensaje[0])])+";"+str(length[int(mensaje[0])])
            dialogo.sendall(str.encode(a.ljust(20)))
            start_p1[int(mensaje[0])] = 0
            start_p2[int(mensaje[0])] = 0
        elif(mensaje[1:]=="points"):
            a = "Puntuación"
            dialogo.sendall(str.encode(a.ljust(20)))
            start_p1[int(mensaje[0])] = 0
            start_p2[int(mensaje[0])] = 0
        elif (mensaje[1:] == "start1"):
            if (start_p1[int(mensaje[0])] == 0 or start_p2[int(mensaje[0])] == 0):
                start_p1[int(mensaje[0])] = 1
                a = "No"
                dialogo.sendall(str.encode(a.ljust(20)))
            elif (start_p1[int(mensaje[0])] == 1 and start_p2[int(mensaje[0])] >=1):
                start_p1[int(mensaje[0])] = 2
                a = "Yes"
                dialogo.sendall(str.encode(a.ljust(20)))
                refreshEnv(drivers[int(mensaje[0])])
            else:
                a = "No"
                dialogo.sendall(str.encode(a.ljust(20)))
        elif (mensaje[1:] == "start2"):
            if (start_p1[int(mensaje[0])] == 0 or start_p2[int(mensaje[0])] == 0):
                start_p2[int(mensaje[0])] = 1
                a = "No"
            #a = setTime()
                dialogo.sendall(str.encode(a.ljust(20)))
            elif (start_p1[int(mensaje[0])] >= 1 and start_p2[int(mensaje[0])] ==1):
                start_p2[int(mensaje[0])] = 2
                a = "Yes"
                dialogo.sendall(str.encode(a.ljust(20)))
                refreshEnv(drivers[int(mensaje[0])])
                #pressf5()
            else:
                a = "No"
                dialogo.sendall(str.encode(a.ljust(20)))
        else:   #el resto de mensajes van a ser inputs
            inputControl_Selenium(mensaje[2:], int(mensaje[1]), drivers[int(mensaje[0])])
            start_p1[int(mensaje[0])] = 0
            start_p2[int(mensaje[0])] = 0
        dialogo.close()
    s.close()
    
if __name__ == '__main__':
    main()